## <center>This Model will Attempt to Predict the Type of Bicycle Used from Trip Data</center>

In [1]:
# Eliminating Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score

In [3]:
# Set up data path
data_path = 'data'

In [4]:
# Load base data to dataframe
all_data_file = 'all_data.csv'
all_df = pd.read_csv(os.path.join(data_path, all_data_file))
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,10/21/16,Citibike to Work,1.15,NaN,9.0,0.15,7.67,Citibike
1,5/28/16,CP BL,7.99,NaN,50.0,0.83,9.59,Specialized
2,9/9/19,To Essex Market,3.71,NaN,31.0,0.52,7.18,Citibike
3,11/4/16,Citibike to Work Midday,1.23,NaN,13.0,0.22,5.68,Citibike
4,6/24/16,Work to W 4th,3.63,NaN,27.0,0.45,8.07,Citibike
...,...,...,...,...,...,...,...,...
352,9/22/16,Work to Westside,4.26,NaN,29.0,0.48,8.81,Citibike
353,8/23/16,Citibike to Work,1.46,NaN,15.0,0.25,5.84,Citibike
354,11/19/16,Down ES UP WS,16.10,1.0,56.0,1.93,8.33,Specialized
355,8/27/19,CP BL x 2,13.66,1.0,49.0,1.82,7.52,Specialized


In [5]:
# Change NaN values to 0 - only occurs on hours column so this is accurate
all_df['Hours'] = all_df['Hours'].fillna(0)
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,10/21/16,Citibike to Work,1.15,0.0,9.0,0.15,7.67,Citibike
1,5/28/16,CP BL,7.99,0.0,50.0,0.83,9.59,Specialized
2,9/9/19,To Essex Market,3.71,0.0,31.0,0.52,7.18,Citibike
3,11/4/16,Citibike to Work Midday,1.23,0.0,13.0,0.22,5.68,Citibike
4,6/24/16,Work to W 4th,3.63,0.0,27.0,0.45,8.07,Citibike
...,...,...,...,...,...,...,...,...
352,9/22/16,Work to Westside,4.26,0.0,29.0,0.48,8.81,Citibike
353,8/23/16,Citibike to Work,1.46,0.0,15.0,0.25,5.84,Citibike
354,11/19/16,Down ES UP WS,16.10,1.0,56.0,1.93,8.33,Specialized
355,8/27/19,CP BL x 2,13.66,1.0,49.0,1.82,7.52,Specialized


In [6]:
# Turning Index into Orig_Index Column
all_df.reset_index(inplace=True)
all_df = all_df.rename(columns = {'index': 'Orig_Index'})
all_df

,Orig_Index,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,0,10/21/16,Citibike to Work,1.15,0.0,9.0,0.15,7.67,Citibike
1,1,5/28/16,CP BL,7.99,0.0,50.0,0.83,9.59,Specialized
2,2,9/9/19,To Essex Market,3.71,0.0,31.0,0.52,7.18,Citibike
3,3,11/4/16,Citibike to Work Midday,1.23,0.0,13.0,0.22,5.68,Citibike
4,4,6/24/16,Work to W 4th,3.63,0.0,27.0,0.45,8.07,Citibike
...,...,...,...,...,...,...,...,...,...
352,352,9/22/16,Work to Westside,4.26,0.0,29.0,0.48,8.81,Citibike
353,353,8/23/16,Citibike to Work,1.46,0.0,15.0,0.25,5.84,Citibike
354,354,11/19/16,Down ES UP WS,16.10,1.0,56.0,1.93,8.33,Specialized
355,355,8/27/19,CP BL x 2,13.66,1.0,49.0,1.82,7.52,Specialized


In [7]:
# Defining fields to use
columns = ['Miles', 'Duration', 'Speed','Type']
# target = ['Type']

In [8]:
# Create Dataframe with desired columns
bike_df = all_df.loc[:,columns].copy()
bike_df

,Miles,Duration,Speed,Type
0,1.15,0.15,7.67,Citibike
1,7.99,0.83,9.59,Specialized
2,3.71,0.52,7.18,Citibike
3,1.23,0.22,5.68,Citibike
4,3.63,0.45,8.07,Citibike
...,...,...,...,...
352,4.26,0.48,8.81,Citibike
353,1.46,0.25,5.84,Citibike
354,16.10,1.93,8.33,Specialized
355,13.66,1.82,7.52,Specialized


In [9]:
# Create Dataframe with desired columns
# Demonstrating different way to do it
# bike_df = all_df[['Miles', 'Duration', 'Speed','Type']]
# bike_df

In [10]:
# Splitting out Features and Target
y = bike_df["Type"]
X = bike_df.drop(columns="Type")

In [11]:
# Splitting out Training and Testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [12]:
# Checking training data shape
X_train.shape

(267, 3)

In [13]:
# Checking testing data shape
X_test.shape

(90, 3)

In [14]:
# Creating the Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

In [15]:
# Training the Model
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [16]:
# Create Predictions
y_pred = classifier.predict(X_test)

In [17]:
# Get accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9888888888888889


In [18]:
# Convert Predictions to DataFrame
pred_df = pd.DataFrame(y_pred)
pred_df.reset_index()
pred_df = pred_df.rename(columns = {0:'Prediction'})
pred_df

,Prediction
0,Specialized
1,Citibike
2,Specialized
3,Specialized
4,Specialized
...,...
85,Citibike
86,Citibike
87,Specialized
88,Citibike


In [19]:
# Convert Tests to DataFrame
test_df = pd.DataFrame(y_test)
test_df

,Type
79,Specialized
173,Citibike
335,Specialized
221,Specialized
81,Specialized
...,...
318,Citibike
237,Citibike
54,Specialized
169,Citibike


In [20]:
# Making index into a column
test_df.reset_index(inplace=True)
test_df = test_df.rename(columns = {'index': 'Orig_Index'})
test_df

,Orig_Index,Type
0,79,Specialized
1,173,Citibike
2,335,Specialized
3,221,Specialized
4,81,Specialized
...,...,...
85,318,Citibike
86,237,Citibike
87,54,Specialized
88,169,Citibike


In [21]:
# Combining the new test and predition dataframes horizontally for comparison
test_pred_df = pd.concat([test_df, pred_df], axis = 1)
test_pred_df

,Orig_Index,Type,Prediction
0,79,Specialized,Specialized
1,173,Citibike,Citibike
2,335,Specialized,Specialized
3,221,Specialized,Specialized
4,81,Specialized,Specialized
...,...,...,...
85,318,Citibike,Citibike
86,237,Citibike,Citibike
87,54,Specialized,Specialized
88,169,Citibike,Citibike


In [22]:
# Identify records where prediction is wrong
pred_errs_df = test_pred_df.loc[test_pred_df['Type'] != test_pred_df['Prediction']]
pred_errs_df

,Orig_Index,Type,Prediction
64,352,Citibike,Specialized


In [23]:
# Merging pred_errs_df with all_df to see details of errors
err_details_df = pd.merge(pred_errs_df, all_df, on=["Orig_Index", "Orig_Index"])
err_details_df

,Orig_Index,Type_x,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type_y
0,352,Citibike,Specialized,9/22/16,Work to Westside,4.26,0.0,29.0,0.48,8.81,Citibike


In [24]:
# Cleaning up err_details_df to eliminate redundancy and improve readability
err_details_df.drop(columns='Type_y', inplace=True)
err_details_df.rename(columns = {'Type_x' : 'Type'}, inplace=True)
err_details_df

,Orig_Index,Type,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed
0,352,Citibike,Specialized,9/22/16,Work to Westside,4.26,0.0,29.0,0.48,8.81
